In [1]:
import pandas as pd
from sklearn import metrics

In [15]:
gold_standard = pd.read_csv('../data/clean_data/gold_standard.csv')

In [9]:
user_input = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,[4000, 5000]]

In [10]:
# binarize avg_rent column
rent_range = user_input[-1]
gold_standard["rent"][(gold_standard["rent"] < rent_range[0]) | (gold_standard["rent"] > rent_range[1])] = 0
gold_standard["rent"][(gold_standard["rent"] >= rent_range[0]) & (gold_standard["rent"] <= rent_range[1])] = 1

/Users/Christine/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/Christine/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
# calculate jaccard similarity
user_input[-1] = 1
for row in range(gold_standard.shape[0]):
    similarity = metrics.jaccard_similarity_score(gold_standard.drop(["similarity", "zip"], axis=1).ix[row,],user_input)
    gold_standard.ix[row,"similarity"] = similarity

In [6]:
gold_standard.sort_values("similarity", ascending=False).reset_index(drop=True).ix[[0,1,2],"zip"]

0    94110
1    94111
2    94102
Name: zip, dtype: int64